In [2]:
# To plot FY-4A BTD plot (T11-T12) by jhlee

import os
os.environ['PROJ_LIB']='/home/jhlee/anaconda3/envs/JUPYTER/share/proj'
from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma


# file path
dir = '/storage1/jhlee/NMSC_2018/FY_4A_AGRI/'


# Read 4 km lat/lon
f = Dataset(dir + 'LatLon_4000m.nc', 'r')
lat = f.variables['Latitude'][:]
lon = f.variables['Longitude'][:]


# Const. Target area
llcrnrlat = 10.
urcrnrlat = 30.
llcrnrlon = 115.
urcrnrlon = 135.


# Read FY-4A channel 12 and 13 digital numbers
hr = ['05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23', '00','01','02','03']
for i in hr:
    year = '2020'
    month = '01'
    day = '12'
    hour = i    
    if (hour == '00' or hour =='01' or hour =='02' or hour =='03'): day = '13'
    sttime = year + month + day + hour
    fname = dir + 'FY4A-_AGRI--_N_DISK_1047E_L1-_FDI-_MULT_NOM_'+sttime+'0000_'+sttime+'1459_4000M_V0001.HDF'
    f = Dataset(fname, 'r')
    
    dn11  = f.variables['NOMChannel12'][:]
    cal11 = f.variables['CALChannel12'][:]
    dn12  = f.variables['NOMChannel13'][:]
    cal12 = f.variables['CALChannel13'][:]

    dn11  = np.array(dn11)
    cal11 = np.array(cal11)
    dn12  = np.array(dn12)
    cal12 = np.array(cal12)

    # TB 11
    idx = np.where(dn11 == 65535.)
    dn11[idx] = -65535.
    tb_11 = cal11[dn11-1]
    idx   = np.where(dn11 == 0)
    tb_11[idx] = np.nan
      
    # TB 12
    idx = np.where(dn12 == 65535.)
    dn12[idx] = -65535.
    tb_12 = cal12[dn12-1]
    idx   = np.where(dn12 == 0)
    tb_12[idx] = np.nan
        
    
# Target area
    btd_11_12 = tb_11 - tb_12
    btd_11_12[lat < llcrnrlat] = np.nan
    btd_11_12[lat > urcrnrlat] = np.nan
    btd_11_12[lon < llcrnrlon] = np.nan
    btd_11_12[lon > urcrnrlon] = np.nan


# Plot BTD (T11-T12) map
    plt.figure(figsize=(8,8))


    map = Basemap(projection='cyl', llcrnrlat= llcrnrlat, urcrnrlat = urcrnrlat,\
                  llcrnrlon = llcrnrlon, urcrnrlon = urcrnrlon, resolution = 'l')

    x, y = map(lon, lat)

    # Taal volcano position
    xi, yi = map(121, 14)
    
#    map.plot(xi, yi, 'm^', markersize=8)
    
    cs = map.pcolormesh(x,y,btd_11_12, shading='flat', cmap=plt.cm.coolwarm)

    map.drawcoastlines(linewidth=2)

    map.drawparallels(np.arange(llcrnrlat,urcrnrlat+1,5.), labels=[True,False,False,False], color='white', linewidth=3.0)
    map.drawmeridians(np.arange(llcrnrlon,urcrnrlon+1,5.), labels=[False,False,False,True], color='white', linewidth=3.0)

    plt.clim(-5.0,5.0) # colorbar range
    cb = map.colorbar(cs, "right")

# show result plot
    plt.title('FY-4A BTD (T11-T12) [K] (%s.%s.%s, %s00 UTC)' %(year, month, day, hour))
    plt.savefig('../plots/FY4A_BTD/FY4A_BTD_11_12_'+sttime+'0000.png', dpi=300)
    plt.show()

KeyboardInterrupt: 

<Figure size 576x576 with 0 Axes>